In [1]:
import numpy as np
import pymysql
import matplotlib.pyplot as plt
import random
import time
import math
from sklearn.linear_model import LogisticRegression

num_train=260864
num_test=261477

In [2]:
user_info={}
f=open('data_format1/user_info_format1.csv')
for line in f:
    l=line.strip()
    if l=='user_id,age_range,gender':
        continue
    [user,age_range,gender]=l.split(',')
    if age_range=='':
        age_range='3'
    if gender=='':
        gender='0'
    user_info[user]=(int(age_range),int(gender))

In [3]:
conn = pymysql.connect(host='localhost', port=3306,user='root',passwd='root',db='cse258',charset='UTF8')
cur = conn.cursor()

In [4]:
def calc_feature(user,seller):
    cmd='SELECT action_type FROM user_log WHERE user_id='+user+' AND seller_id='+seller
    cur.execute(cmd)
    result=cur.fetchall()
    tot_len=len(result)
    lens=[0,0,0,0]
    for datum in result:
        lens[datum[0]]+=1
    lens.append(tot_len)
    return lens

In [5]:
X=[]
y=[]
f_train=open('data_format1/train_format1.csv')
cnt=0
print_every=5000
for line in f_train:
    l=line.strip()
    if l=='user_id,merchant_id,label':
        continue
    [user,seller,this_y]=l.split(',')
    fe=calc_feature(user,seller)
    info=user_info[user]
    X.append([1,info[0],info[1],fe[0],fe[1],fe[2],fe[3],fe[4]])
    y.append(int(this_y))
    if cnt>0 and (cnt==1 or cnt%print_every==0 or cnt==num_train):
        print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),
              'finished '+str(cnt)+' pairs')
    cnt+=1

X=np.array(X)
y=np.array(y)
print(X.shape)
print(y.shape)
np.save('data/x_userinfo_ratio.npy', X)
np.save('data/y.npy', y)

2017-11-25 12:17:16 finished 1 pairs
2017-11-25 12:21:13 finished 5000 pairs
2017-11-25 12:25:33 finished 10000 pairs
2017-11-25 12:29:50 finished 15000 pairs
2017-11-25 12:33:57 finished 20000 pairs
2017-11-25 12:38:10 finished 25000 pairs
2017-11-25 12:42:24 finished 30000 pairs
2017-11-25 12:46:36 finished 35000 pairs
2017-11-25 12:50:53 finished 40000 pairs
2017-11-25 12:55:08 finished 45000 pairs
2017-11-25 12:59:25 finished 50000 pairs
2017-11-25 13:03:40 finished 55000 pairs
2017-11-25 13:08:01 finished 60000 pairs
2017-11-25 13:12:24 finished 65000 pairs
2017-11-25 13:16:48 finished 70000 pairs
2017-11-25 13:21:13 finished 75000 pairs
2017-11-25 13:25:38 finished 80000 pairs
2017-11-25 13:29:56 finished 85000 pairs
2017-11-25 13:34:15 finished 90000 pairs
2017-11-25 13:38:38 finished 95000 pairs
2017-11-25 13:42:59 finished 100000 pairs
2017-11-25 13:47:18 finished 105000 pairs
2017-11-25 13:51:39 finished 110000 pairs
2017-11-25 13:56:00 finished 115000 pairs
2017-11-25 14:00:

In [11]:
X_test=[]
f_test=open('data_format1/test_format1.csv')
cnt=0
print_every=5000
for line in f_test:
    l=line.strip()
    if l=='user_id,merchant_id,prob':
        continue
    [user,seller,_]=l.split(',')
    fe=calc_feature(user,seller)
    info=user_info[user]
    X_test.append([1,info[0],info[1],fe[0],fe[1],fe[2],fe[3],fe[4]])
    if cnt>0 and (cnt==1 or cnt%print_every==0 or cnt==num_test):
        print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),
              'finished '+str(cnt)+' pairs')
    cnt+=1

X_test=np.array(X_test)
print(X_test.shape)
np.save('data/x_userinfo_ratio_test.npy', X_test)

2017-11-25 16:02:55 finished 1 pairs
2017-11-25 16:07:12 finished 5000 pairs
2017-11-25 16:11:44 finished 10000 pairs
2017-11-25 16:16:07 finished 15000 pairs
2017-11-25 16:20:30 finished 20000 pairs
2017-11-25 16:24:51 finished 25000 pairs
2017-11-25 16:29:17 finished 30000 pairs
2017-11-25 16:33:49 finished 35000 pairs
2017-11-25 16:38:10 finished 40000 pairs
2017-11-25 16:42:29 finished 45000 pairs
2017-11-25 16:46:47 finished 50000 pairs
2017-11-25 16:51:06 finished 55000 pairs
2017-11-25 16:55:29 finished 60000 pairs
2017-11-25 16:59:56 finished 65000 pairs
2017-11-25 17:04:19 finished 70000 pairs
2017-11-25 17:08:36 finished 75000 pairs
2017-11-25 17:13:43 finished 80000 pairs
2017-11-25 17:21:35 finished 85000 pairs
2017-11-25 17:25:56 finished 90000 pairs
2017-11-25 17:30:10 finished 95000 pairs
2017-11-25 17:34:29 finished 100000 pairs
2017-11-25 17:38:47 finished 105000 pairs
2017-11-25 17:43:07 finished 110000 pairs
2017-11-25 17:47:32 finished 115000 pairs
2017-11-25 17:51:

In [14]:
cur.close()
conn.close()